# ML Pipeline for "Forecasting Air Quality with Amazon SageMaker DeepAR

In this example, we are going to build a ML Pipeline to automate air quality forecasting application with [AWS Step Functions Data Science SDK](https://aws-step-functions-data-science-sdk.readthedocs.io). 

## ML Pipeline

### Outcome
* Create the flow for ML process for air quality forcasting build/train/deploy
* Create simple retrain flow

### Design
* Use Step Functions Data Science SDK to orchestrate the ML flow
* Use SageMaker Processing to do data preprocessing, especially,
 * A common Docker image will be build for data retrieving (interact with Amazon Athena) and data/feature engineering
* Use SageMaker Processing to do Model Evaluation
* A scheduled job mechanism will be used to do model retraining.

### Implementation

#### Initialize

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install -qU awscli boto3 "sagemaker==1.71.0"
!{sys.executable} -m pip install -qU "stepfunctions==1.1.0"
!{sys.executable} -m pip show sagemaker stepfunctions

In [ ]:
import uuid
import time
import boto3
import stepfunctions
from stepfunctions import steps
from stepfunctions.inputs import ExecutionInput
from stepfunctions.steps import (
    Chain,
    ChoiceRule,
    ModelStep,
    ProcessingStep,
    TrainingStep,
    TransformStep
)
from stepfunctions.workflow import Workflow

import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.processing import ProcessingInput, ProcessingOutput

sagemaker_session = sagemaker.Session()

region = boto3.session.Session().region_name
role = get_execution_role()

session = boto3.Session()
account_id = session.client('sts').get_caller_identity().get('Account')
bucket_name = f'{account_id}-openaq-lab'

#### Create Docker Image for SageMaker Processing

Define your own processing container and install related dependencies.

Below, you talk through how to create a processing container, and how to use a `ScriptProcessor` to run your own code within a container. Create a container support data preprocessing, feature engineering and model evaluation. 

In [ ]:
# create a subfolder for docker 
!mkdir -p docker

Below is the Dockerfile to create processing container. Install PyAthena, pandas and GeoPandas into it. You can install your own dependencies.

In [ ]:
%%writefile docker/Dockerfile

FROM python:3.7-slim-buster
    
COPY ./sql /opt/ml/processing/sql
    
RUN pip install pandas numpy geopandas scikit-learn fsspec s3fs boto3

ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

This block of code buils the container using the docker command, creates an Amazon Elastic Container Registry (Amazon ECR) repository, and pushes the image to Amazon ECR

In [ ]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'aq-forecasting-processing-container'
tag = ':latest'

uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'
processing_repository_uri = f'{account_id}.dkr.ecr.{region}.{uri_suffix}/{ecr_repository + tag}'


In [ ]:
processing_repository_uri

In [ ]:
# @todo consider using CFN template to create ECR repo and only manage the docker image build and push.
!docker build -t $ecr_repository docker

In [ ]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

Below cell writes a file `preprocessing.py`, which contains the pre-processing script. You can update the script, and rerun the cell to overwrite `preprocessing.py`. You run this as a processing job in the next cell. In this script, the actions will be done:

* Create Athena table with external source - OpenAQ
* Query OpenAQ data 
* Feature engineering on the dataset
* Split and store the data on S3 buckets.

Upload the pre processing script.

In [ ]:
PREPROCESSING_SCRIPT_LOCATION = "preprocessing.py"
input_code = sagemaker_session.upload_data(
    PREPROCESSING_SCRIPT_LOCATION,
    bucket = bucket_name,
    key_prefix = "preprocessing/code",
)

S3 locations of preprocessing output with training, test & all features.

In [ ]:
output_data = f"s3://{bucket_name}/preprocessing/output"

The `ScriptProcessor` class lets you run a command inside the container, which you can use to run your own script.

In [ ]:
from sagemaker.processing import ScriptProcessor

preprocessing_processor = ScriptProcessor(
    command = ['python3'],
    image_uri = processing_repository_uri,
    role = role,
    instance_count = 1,
    instance_type = 'ml.m5.xlarge',
    max_runtime_in_seconds = 1200
)

### Create the ProcessingStep
We will now create the [ProcessingStep](https://aws-step-functions-data-science-sdk.readthedocs.io/en/stable/sagemaker.html#stepfunctions.steps.sagemaker.ProcessingStep) that will launch a SageMaker Processing Job.

This step will use ScriptProcessor as defined in previous steps along with the inputs and outputs objects that are defined in the below steps.

In [ ]:
inputs = [
    ProcessingInput(
        source = input_code,
        destination = "/opt/ml/processing/input/code",
        input_name = "code"
    )
]

outputs = [
    ProcessingOutput(
        source = "/opt/ml/processing/output/all",
        destination = f"{output_data}/all",
        output_name = "all_data"
    ),
    ProcessingOutput(
        source = "/opt/ml/processing/output/train",
        destination = f"{output_data}/train",
        output_name = "train_data"
    ),
    ProcessingOutput(
        source = "/opt/ml/processing/output/test",
        destination = f"{output_data}/test",
        output_name = "test_data"
    )
]

In [ ]:
execution_input = ExecutionInput(
    schema = {
        "PreprocessingJobName": str,
        "ToDoHPO": bool,
        "ToDoTraining": bool,
        "TrainingJobName": str,
        "TuningJobName": str,
        "ModelName": str,
        "EndpointName": str,
        "EvaluationProcessingJobName": str
    }
)

#### Create the ProcessingStep

In [ ]:
processing_step = ProcessingStep(
    "AirQualityForecasting Pre-processing Step",
    processor = preprocessing_processor,
    job_name = execution_input["PreprocessingJobName"],
    inputs = inputs,
    outputs = outputs,
    container_arguments = ["--split-days", "30"],
    container_entrypoint = ["python3", "/opt/ml/processing/input/code/preprocessing.py"]
)

### HyperParameter Tuning

Setup tuning step and use choice state to decide whether we should do HPO.

### Training

We create a DeepAR instance, which we will use to run a training job. This will be used to create a TrainingStep for the workflow.

In [ ]:
from sagemaker.tuner import HyperparameterTuner, ContinuousParameter, IntegerParameter
from sagemaker.model import Model


In [ ]:
image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")

In [ ]:
output_path = f's3://{bucket_name}/tuning/output'

ml_instance_type = 'ml.c5.9xlarge'

tunning_estimator = sagemaker.estimator.Estimator(
        sagemaker_session = sagemaker_session,
        image_name = image_name,
        role = role,
        train_instance_count = 1,
        train_instance_type = ml_instance_type,
        base_job_name = 'deepar-openaq-demo',
        output_path = output_path
)




#### Set static hyperparameters
The static parameters are the ones we know to be the best based on previously run HPO jobs, as well as the non-tunable parameters like prediction length and time frequency that are set according to requirements.

In [ ]:
hpo = dict(
    time_freq= '1H'
    ,early_stopping_patience= 40
    ,prediction_length= 48
    ,num_eval_samples= 10

    # default quantiles [0.1, 0.2, 0.3, ..., 0.9] is used
    #,test_quantiles= quantiles
    
    # not setting these since HPO will use range of values
    #,epochs= 400
    #,context_length= 3
    #,num_cells= 157
    #,num_layers= 4
    #,dropout_rate= 0.04
    #,embedding_dimension= 12
    #,mini_batch_size= 633
    #,learning_rate= 0.0005
)

#### Set hyper-parameter ranges
The hyperparameter ranges define the parameters we want the runer to search across.

> Explore: Look in the [user guide](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html) for DeepAR and add the recommended ranges for `embedding_dimension` to the below.

In [ ]:
hpo_ranges = dict(
    epochs= IntegerParameter(1, 1000)
    ,context_length= IntegerParameter(7, 48)
    ,num_cells= IntegerParameter(30,200)
    ,num_layers= IntegerParameter(1,8)
    ,dropout_rate= ContinuousParameter(0.0, 0.2)
    ,embedding_dimension= IntegerParameter(1, 50)
    ,mini_batch_size= IntegerParameter(32, 1028)
    ,learning_rate= ContinuousParameter(.00001, .1)
)

#### Create HPO tunning job step
Once we have the HPO tuner defined, we can define the tuning step.

In [ ]:
tuning_estimator.set_hyperparameters(**hpo)

hpo_tuner = tuning_estimator(
    estimator= estimator, 
    objective_metric_name = 'train:final_loss',
    objective_type = 'Minimize',
    hyperparameter_ranges = hpo_ranges,
    max_jobs = 2,
    max_parallel_jobs = 1
)

hpo_data = dict(
    train = f"{output_data}/train",
    test = f"{output_data}/test"
)
# as long as HPO is selected, wait for completion.
tuning_step = steps.TuningStep(
    "Tuning Step",
    tuner = hpo_tuner,
    job_name = execution_input["TuningJobName"],
    data = hpo_data,
    wait_for_completion = True
)

#### Setup the training job step

In [ ]:
output_path = f's3://{bucket_name}/training/output'
training_estimator = sagemaker.estimator.Estimator(
        sagemaker_session = sagemaker_session,
        image_name = image_name,
        role = role,
        train_instance_count = 1,
        train_instance_type = ml_instance_type,
        base_job_name = 'deepar-openaq-demo',
        output_path = output_path
)


In [ ]:
# best hyper parameters for tuning
hpo = dict(
    time_freq= '1H'
    ,early_stopping_patience= 40
    ,prediction_length= 48
    ,num_eval_samples= 10
    #,test_quantiles= quantiles
    ,epochs= 400
    ,context_length= 3
    ,num_cells= 157
    ,num_layers= 4
    ,dropout_rate= 0.04
    ,embedding_dimension= 12
    ,mini_batch_size= 633
    ,learning_rate= 0.0005
)
estimator.set_hyperparameters(**hpo)

In [ ]:
# use all the features for training.
data = dict(train = f"{output_data}/all/all_features.json")
training_step = steps.TrainingStep(
    "Training Step",
    estimator = estimator,
    data = data,
    job_name = execution_input["TrainingJobName"],
    wait_for_completion = True
)


#### Create a Model Step

In the following cell, we define a model step that will create a model in Amazon SageMaker using the artifacts created during the TrainingStep. See  [ModelStep](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/sagemaker.html#stepfunctions.steps.sagemaker.ModelStep) in the AWS Step Functions Data Science SDK documentation to learn more.

The model creation step typically follows the training step. The Step Functions SDK provides the [get_expected_model](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/sagemaker.html#stepfunctions.steps.sagemaker.TrainingStep.get_expected_model) method in the TrainingStep class to provide a reference for the trained model artifacts. Please note that this method is only useful when the ModelStep directly follows the TrainingStep.

In [ ]:
model_step = steps.ModelStep(
    "Save Model",
    model = training_step.get_expected_model(),
    model_name = execution_input["ModelName"],
    result_path = "$.ModelStepResults"
)

existing_model_name = f"aqf-model-{uuid.uuid1().hex}"
existing_model = Model(
    model_data = f"s3//{bucket_name}/sagemaker/model/model.tar.gz",
    image = image_name,
    role = role,
    name = existing_model_name
)
existing_model_step = steps.ModelStep(
    "Existing Model",
    model = existing_model,
    model_name = execution_input["ModelName"],
    result_path = "$.ModelStepResults"
)

### Create an Endpoint Configuration Step
In the following cell we create an endpoint configuration step. See [EndpointConfigStep](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/sagemaker.html#stepfunctions.steps.sagemaker.EndpointConfigStep) in the AWS Step Functions Data Science SDK documentation to learn more.

In [ ]:
endpoint_config_step = steps.EndpointConfigStep(
    "Create Model Endpoint Config",
    endpoint_config_name = execution_input["ModelName"],
    model_name = execution_input["ModelName"],
    initial_instance_count = 1,
    instance_type = 'ml.c5.xlarge'
)

### Update the Model Endpoint Step
In the following cell, we create the Endpoint step to deploy the new model as a managed API endpoint, updating an existing SageMaker endpoint if our choice state is sucessful.

In [ ]:
endpoint_step = steps.EndpointStep(
    "Update Model Endpoint",
    endpoint_name = execution_input["EndpointName"],
    endpoint_config_name = execution_input["ModelName"],
    update = False
)

#### Model Evaluation

Run model evaluation.

In [ ]:
# @todo

#### Setup workflow process

Create `Fail` state to mark the workflow failed in case any of the steps fail.

In [ ]:
failed_state_sagemaker_processing_failure = stepfunctions.steps.states.Fail(
    "ML Workflow failed", cause = "SageMakerProcessingJobFailed"
)

In [ ]:
training_path = Chain([training_step, model_step, endpoint_config_step, endpoint_step])
deploy_existing_model_path = Chain([existing_model_step, endpoint_config_step, endpoint_step])

#### Choice state

Now, we need to setup choice state for choose HPO / Training or not. See *Choice Rules* in the [AWS Step Functions Data Science SDK documentation](https://aws-step-functions-data-science-sdk.readthedocs.io) .

In [ ]:
hpo_choice = Choice(
    "To do HPO?"
)
training_choice = Choice(
    "To do Model Training?"
)
hpo_choice.add_choice(
    rule = ChoiceRule.BooleanEquals(variable = execution_input['ToDoHPO'], value = True),
    next_step = hpo_step
)
hpo_choice.add_choice(
    rule = ChoiceRule.BooleanEquals(variable = execution_input['ToDoHPO'], value = False),
    next_step = training_choice
)
training_choice.add_choice(
    rule = ChoiceRule.BooleanEquals(variable = execution_input['ToDoTraining'], value = True),
    next_step = training_path
)
training_choice.add_choice(
    rule = ChoiceRule.BooleanEquals(variable = execution_input['ToDoTraining'], value = False),
    next_step = deploy_existing_model_path
)

#### Add the Error handling in the workflow

In [ ]:
catch_state_processing = stepfunctions.steps.states.Catch(
    error_equals = ["States.TaskFailed"],
    next_step = failed_state_sagemaker_processing_failure   
)
processing_step.add_catch(catch_state_processing)
training_step.add_catch(catch_state_processing)

#### Workflow Role

In [ ]:
workflow_execution_role = "arn:aws:iam::593380422482:role/StepFunctionsWorkflowExecutionRole"

#### Create StepFunctions Workflow execution Input schema

In [ ]:
preprocessing_job_name = f"aqf-preprocessing-{uuid.uuid1().hex}"
tuning_job_name = f"aqf-tuning-{uuid.uuid1().hex}"
training_job_name = f"aqf-training-{uuid.uuid1().hex}"
model_job_name = f"aqf-model-{uuid.uuid1().hex}"
endpoint_job_name = f"aqf-endpoint-{uuid.uuid1().hex}"
evaluation_job_name = f"aqf-evaluation-{uuid.uuid1().hex}"

In [ ]:
len(preprocessing_job_name)

### Create and execute the workflow

In [ ]:
workflow_graph = Chain([processing_step, hpo_choice, training_choice])
workflow = Workflow(
    name = "AirQualityForecastingWorkflow",
    definition = workflow_graph,
    role = workflow_execution_role
)
workflow.create()
# update() to ensure existing workflow can get updated as create() just return ARN for the existing one.
workflow.update(definition = workflow_graph) 

# execute workflow
execution = workflow.execute(
    inputs = {
        "PreprocessingJobName": preprocessing_job_name,
        "ToDoHPO": False,
        "ToDoTraining": False,
        "TrainingJobName": training_job_name,
        "TuningJobName": tuning_job_name,
        "ModelName": model_job_name,
        "EndpointName": endpoint_job_name,
        "EvaluationProcessingJobName": evaluation_job_name
    }
)
execution_output = execution.get_output(wait = True)

In [ ]:
execution.render_progress()